## Binary Indexed Tree, build O(n), cal sum/update O(logn)

In [34]:
class NumArray:

    def __init__(self, nums):
        self.origin = nums
        self.size = len(nums)
        # O(nlogn) build via add
        # self.sum_arr = [0] * (self.size + 1)
        # for i in range(self.size):
            # self.add(i, self.origin[i])
        # O(n) build
        self.sum_arr = [0] + self.origin
        for i in range(1, self.size + 1):
            j = i + (i & (-i))
            if j < self.size + 1:
                self.sum_arr[j] += self.sum_arr[i]
        # print (self.sum_arr)
        
    def add(self, i, val):
        i += 1
        while i <= self.size:
            self.sum_arr[i] += val
            i += i & (-i)
        
    def update(self, i: int, val: int) -> None:
        self.add(i, val - self.origin[i])
        self.origin[i] = val
        
    def cal_sum (self, i):
        i += 1
        res = 0
        while i > 0:
            res += self.sum_arr[i]
            i -= i & (-i)
        return res
        
    def sumRange(self, i: int, j: int) -> int:
        return self.cal_sum(j) - self.cal_sum(i - 1)

In [35]:
a = NumArray([1, 3, 5])

In [36]:
a.sumRange(0, 2) 

9

In [37]:
a.update(1, 2)

In [38]:
a.sumRange(0, 2)

8

## SegmentTree via TreeNode, build O(n), update/sum O(logn)

In [3]:
class SegmentTreeNode():
    def __init__(self, start, end, total=0):
        self.start, self.end = start, end
        self.total = total
        self.left, self.right = None, None

class SegmentTree():
    def __init__(self, nums):
        self.root = self.build_tree(0, len(nums) - 1, nums)
    
    def build_tree(self, start, end, nums):
        if start > end: return
        if start == end:
            return SegmentTreeNode(start, end, nums[start])
        node = SegmentTreeNode(start, end)
        mid = start + (end - start) // 2
        node.left = self.build_tree(start, mid, nums)
        node.right = self.build_tree(mid + 1, end, nums)
        if node.left:
            node.total += node.left.total
        if node.right:
            node.total += node.right.total

        return node
    
    def update(self, node, i, val):
        if node.start == i and node.end == i:
            node.total = val
            return
        mid = node.start + (node.end - node.start) // 2
        if i <= mid:
            self.update(node.left, i, val)
        else:
            self.update(node.right, i, val)
        node.total = 0
        if node.left:
            node.total += node.left.total
        if node.right:
            node.total += node.right.total
    
    def sumRange(self, node, i, j):
        if not node:
            return 0
        if i > node.end or i < node.start or j > node.end or j < node.start:
            return
        if node.start == i and node.end == j:
            return node.total
        mid = node.start + (node.end - node.start) // 2
        if j <= mid:
            return self.sumRange(node.left, i, j)
        elif i > mid:
            return self.sumRange(node.right, i, j)
        else:
            return self.sumRange(node.left, i, mid) + self.sumRange(node.right, mid + 1, j)

class NumArray:

    def __init__(self, nums):
        self.SegmentTree = SegmentTree(nums)

    def update(self, i: int, val: int) -> None:
        self.SegmentTree.update(self.SegmentTree.root, i, val)

    def sumRange(self, i: int, j: int) -> int:
        return self.SegmentTree.sumRange(self.SegmentTree.root, i, j)


In [4]:
a = NumArray([1, 3, 5])

In [5]:
a.sumRange(0, 2) 

9

In [6]:
a.update(1, 2)

In [7]:
a.sumRange(0, 2)

8

## Segment Tree via Array, build O(n), update/sum O(logn)

In [18]:
class NumArray:

    def __init__(self, nums):
        self.size = len(nums)
        self.tree_arr = self.size * [0] + nums
        for i in range(self.size - 1, 0, -1):
            # i's sons are 2i, 2i + 1
            self.tree_arr[i] = self.tree_arr[i << 1] + self.tree_arr[i << 1 | 1]

    def update(self, i: int, val: int) -> None:
        i += self.size
        self.tree_arr[i] = val
        while i > 1:
            # if i is odd, left is i - 1, if i is even, right is i + 1
            self.tree_arr[i >> 1] = self.tree_arr[i] + self.tree_arr[i ^ 1]
            i >>= 1

    def sumRange(self, i: int, j: int) -> int:
        l = i + self.size
        r = j + self.size
        res = 0
        while l <= r:
            if l & 1:
                res += self.tree_arr[l]
                l += 1
            l >>= 1
            if not r & 1:
                res += self.tree_arr[r]
                r -= 1
            r >>= 1
        return res

In [19]:
a = NumArray([1, 3, 5])

In [20]:
a.sumRange(0, 2) 

9

In [21]:
a.update(1, 2)

In [22]:
a.sumRange(0, 2)

8